In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import numpy as np
import scanpy as sc
import cellrank as cr
import scvelo as scv
from tueplots import bundles
from tueplots import axes
import anndata as ad
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [3]:
plt.rcParams.update(**bundles.neurips2022(usetex=False, nrows=2), **axes.lines(base_width=0.5), **{"figure.dpi": 250})

scv.set_figure_params('scvelo', fontsize=20, color_map='viridis', dpi=400)  # for beautified visualization

cr.logging.print_versions()

cellrank==1.5.1 scanpy==1.9.1 anndata==0.8.0 numpy==1.23.5 numba==0.56.4 scipy==1.9.3 pandas==1.5.2 pygpcca==1.0.4 scikit-learn==1.1.3 statsmodels==0.13.5 scvelo==0.2.5 pygam==0.8.0 matplotlib==3.6.2 seaborn==0.12.1


In [4]:
adata = scv.datasets.gastrulation()
print("data loaded")

adata_25 = sc.pp.subsample(adata[adata.obs['stage'] == 'E8.25', :], n_obs=1000, copy=True)
adata_5 = sc.pp.subsample(adata[adata.obs['stage'] == 'E8.5', :], n_obs=1000, copy=True) 

del adata

data loaded


In [5]:
scv.tl.recover_dynamics(adata_25, n_jobs=7)
scv.tl.velocity(adata_25, mode='dynamical')

recovering dynamics (using 7/8 cores)


  0%|          | 0/53801 [00:00<?, ?gene/s]

: 

: 

In [ ]:
scv.tl.recover_dynamics(adata_5, n_jobs=7)
scv.tl.velocity(adata_5, mode='dynamical')

In [11]:
adata = ad.concat([adata_25, adata_5])
print(adata)

AnnData object with n_obs × n_vars = 2000 × 53801
    obs: 'barcode', 'sample', 'stage', 'sequencing.batch', 'theiler', 'doub.density', 'doublet', 'cluster', 'cluster.sub', 'cluster.stage', 'cluster.theiler', 'stripped', 'celltype', 'colour', 'umapX', 'umapY', 'haem_gephiX', 'haem_gephiY', 'haem_subclust', 'endo_gephiX', 'endo_gephiY', 'endo_trajectoryName', 'endo_trajectoryDPT', 'endo_gutX', 'endo_gutY', 'endo_gutDPT', 'endo_gutCluster', 'cell_velocyto_loom'
    obsm: 'X_pca', 'X_umap'
    layers: 'spliced', 'unspliced'


: 

In [ ]:
from neural_ot.utils import get_neural_dual_from_ckpt, mmd_rbf, sinkhorn_loss

In [ ]:
adata.layers["velocity"] = -1 * adata.layers["velocity"].copy()

In [ ]:
print(sinkhorn_loss(adata[adata.obs["stage"] == "8.25"].X + adata[adata.obs["stage"] == "8.25"].layers["velocity"], adata[adata.obs["stage"] == "8.5"].X))

In [ ]:
vk = cr.kernels.VelocityKernel(adata)
vk.compute_transition_matrix()

In [ ]:
g = cr.estimators.GPCCA(vk)

In [ ]:
g.compute_macrostates(n_states=10, cluster_key="celltype")

In [ ]:
g.compute_terminal_states(method="top_n", n_states=10)
scv.set_figure_params('scvelo', transparent=True, fontsize=20, color_map='viridis')
g.plot_terminal_states(discrete=True, basis="umap", legend_loc="right", legend_fontweight='normal', legend_fontsize='12', dpi=250)

In [ ]:
g.set_terminal_states_from_macrostates(
    [
        "Acinar_1", "Acinar_2", "Ductal", "Alpha", "Beta_1", "Beta_2", "Delta", "Epsilon"
    ]
)
g.compute_absorption_probabilities(solver="gmres", use_petsc=True, tol=1e-12, preconditioner='ilu', time_to_absorption='all')
g.plot_absorption_probabilities(same_plot=False, basis="umap", perc=[0, 99], ncols=2)

In [ ]:
cr.pl.aggregate_absorption_probabilities(
    adata,
    mode='heatmap',
    lineages=
    [
         "Acinar_1", "Acinar_2", "Ductal", "Alpha", "Beta_1", "Beta_2", "Delta", "Epsilon"
    ],
    cluster_key='celltype',
    clusters=['Acinar', 'Tip', 'Ductal', 'Ngn3 High early','Ngn3 High late', 'Fev+ Alpha', 'Alpha', 'Fev+ Beta', 'Beta', 'Fev+ Delta', 'Delta', 'Fev+ Epsilon', 'Epsilon'],
    figsize=(17, 6),
    title="",
    #save="transprobs_pancreas1415_trajectorynet.png"
)

In [ ]:
adata.uns["velocity_graph"] = vk.transition_matrix
scv.pl.velocity_embedding_stream(adata,
                                 basis="umap",
                                 title="",
                                 legend_loc="none",
                                 smooth=0.5, 
                                 #save="velstream_pancreas1415_trajectorynet.png"
                                )

In [ ]:
adata.layers["Ms"] = adata.layers["pca"].copy()
scv.tl.velocity_confidence(adata)

In [ ]:
scv.set_figure_params('scvelo', fontsize=20)
scv.settings.presenter_view = False

scv.settings.set_figure_params('scvelo', dpi_save=800, dpi=400, transparent=True, fontsize=20, color_map='viridis')

In [ ]:
scv.pl.scatter(adata, color=['velocity_confidence'],
               perc=[2, 98],
               cmap='gnuplot',
               vmid=0.75,
               rescale_color=[0.5, 1.0],
               size=15,
               color_map="none",
               title="",
               #save="velconf_pancreas1415_trajectorynet.png"
              )